In [4]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD')
print(os.getcwd())
import pandas as pd
import numpy as np
import datetime

import plotly.express as px

df=pd.read_csv('modeled_data.csv')

df['date']=pd.to_datetime(df['date'])
df['week']=df['date'].dt.strftime('%U').astype(int)

df=df.sort_values(['store_nbr','date'],ascending=[True,True])

df.dtypes

/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD


date                 datetime64[ns]
store_nbr                     int64
sales                       float64
promotion_count               int64
city                         object
state                        object
store_type                   object
cluster                       int64
holiday_type                 object
locale                       object
locale_name                  object
description                  object
transferred                  object
year                          int64
month                         int64
day_of_mnth                   int64
day_of_week                  object
daynum_of_week                int64
mnth_name                    object
st_B                          int64
st_C                          int64
st_D                          int64
st_E                          int64
min_non_zero_date            object
starting_dt                  object
days_from_start             float64
active                       object
yr_2013                     

In [5]:
cluster_cnt=df.groupby(['cluster'])['store_nbr'].nunique().reset_index().rename(columns={'store_nbr':'store_cnt'})
cluster_cnt['cluster_type']='multi'
cluster_cnt['cluster_type']=np.where((cluster_cnt['store_cnt']==1),'single',cluster_cnt['cluster_type'])
#cluster_cnt['cluster_type']=np.where((cluster_cnt['store_cnt']==2),'dual',cluster_cnt['cluster_type'])
cluster_cnt


df=pd.merge(df,cluster_cnt,on=['cluster'],how='inner')

In [6]:
##handiling single cluster cases

sc=df[df['cluster_type']=='single']

sc=sc[['date','store_nbr','sales','outliers','scores','hol_flg','cluster','holiday_type','description']]

sc.sort_values(['store_nbr','date'],ascending=[True,True],inplace=True)

sc.reset_index(drop=True,inplace=True)



sc_store_list=list(set(sc.store_nbr.to_list()))

sc_store_list

odf=pd.DataFrame()

for x in sc_store_list:
  
  store_df=sc[sc['store_nbr']==x]

  outlier_lst=store_df['outliers'].to_list()
  dt_lst=store_df['date'].to_list()

  od=[]

  for i in range(len(outlier_lst)-2):
    if outlier_lst[i]=='N':
      continue
    else:
      if outlier_lst[i+1]=='Y':
        if outlier_lst[i+2]=='Y':
          od.append(dt_lst[i])
          od.append(dt_lst[i+1])
          od.append(dt_lst[i+2])
  
  ol=[]
  for y in od:
    if y not in ol:
      ol.append(y)

 
  r=store_df[store_df['date'].isin(ol)]
  

  odf=odf.append(r)



odf['outlier_type']='weak outlier with holiday'

odf.loc[(odf.holiday_type.isnull()),'outlier_type']='outliers'

odf=odf[['date','store_nbr','outlier_type']]

sc=pd.merge(sc,odf,on=['date','store_nbr'],how='left')
sc.loc[sc.outlier_type.isnull(),'outlier_type']='Not outlier'

sc.to_csv('single_store_output.csv',index=False)

In [8]:
###handling multi cluster cases


mc=df[df['cluster_type']!='single']

mc=mc[['date','store_nbr','sales','year','month','daynum_of_week','promotion_count','outliers','scores','hol_flg','cluster','holiday_type','description','week']]

mc.sort_values(['store_nbr','date'],ascending=[True,True],inplace=True)

mc.reset_index(drop=True,inplace=True)

cluster_list=list(set(mc['cluster'].to_list()))
store_list=list(set(mc['store_nbr'].to_list()))


bench_mnth=mc.groupby(['year','month','cluster'])['sales'].mean().reset_index().rename(columns={'sales':'cluster_sales_mnth'})

bench_daily=mc.groupby(['date','cluster'])['sales'].mean().reset_index().rename(columns={'sales':'cluster_sales_day'})

store_mnth=mc.groupby(['year','month','store_nbr'])['sales'].mean().reset_index().rename(columns={'sales':'store_sales_mnth'})


mc=pd.merge(mc,bench_mnth,on=['year','month','cluster'],how='inner')


mc=pd.merge(mc,bench_daily,on=['date','cluster'],how='inner')

mc=pd.merge(mc,store_mnth,on=['year','month','store_nbr'],how='inner')

mc['cluster_deviation']=mc['cluster_sales_day']/mc['cluster_sales_mnth']

mc['store_deviation']=mc['sales']/mc['store_sales_mnth']

mc

mcdf=pd.DataFrame()

for x in store_list:
  
  store_df=mc[mc['store_nbr']==x]

  outlier_lst=store_df['outliers'].to_list()
  dt_lst=store_df['date'].to_list()

  dd=[]

  for i in range(len(outlier_lst)-2):
    if outlier_lst[i]=='N':
      continue
    else:
      if outlier_lst[i+1]=='Y':
        if outlier_lst[i+2]=='Y':
          dd.append(dt_lst[i])
          dd.append(dt_lst[i+1])
          dd.append(dt_lst[i+2])
  
  ol=[]
  for y in dd:
    if y not in ol:
      ol.append(y)



 
  r=store_df[store_df['date'].isin(ol)]
  

  mcdf=mcdf.append(r)




mcdf['deviation_diff']=abs(mcdf['store_deviation']-mcdf['cluster_deviation'])
mcdf=mcdf[['date','store_nbr','holiday_type','description','deviation_diff']]

mcdf['outlier_type']='Weak- holiday and minor deviation from cluster'

cr1=mcdf['holiday_type'].isnull()
cr2=mcdf['deviation_diff']>=0.1


cr3=cr1&cr2

mcdf.loc[cr3,'outlier_type']='Strong - no holiday and deviates from cluster'

cr4=~cr1

cr5=cr4&cr2
mcdf.loc[cr5,'outlier_type']='Med strong - holiday but deviates from cluster'

cr6=~cr2
cr7=cr6&cr1

mcdf.loc[cr7,'outlier_type']='Med Weak - no holiday but low deviation from cluster'

mcdf=mcdf[['date','store_nbr','outlier_type']]

mc_final=pd.merge(mc,mcdf,on=['date','store_nbr'],how='left')

mc_final.loc[mc_final.outlier_type.isnull(),'outlier_type']='Not outlier'

mc_final.to_csv('multi_store_output.csv',index=False)